# **Content Based Recommendation**







In [1]:
!pip install rake_nltk
import nltk
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [3]:
dataset = pd.read_csv("/content/anime.csv")
synopsis = pd.read_csv("/content/anime_with_synopsis.csv")

In [4]:
display(dataset.shape)
display(synopsis.shape)
dataset.head()

(17562, 35)

(16214, 5)

,MAL_ID,Name,Score,Genders,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [5]:
#Merge the two dataset
dataset = synopsis.merge(dataset, how='left', on='MAL_ID')
dataset.shape

(16214, 39)

In [6]:
dataset['Type'].value_counts()

TV         4996
Movie      3031
OVA        2600
Special    2195
ONA        1886
Music      1469
Unknown      37
Name: Type, dtype: int64

In [7]:
#Filter out the 'Music', 'Unknown', 'OVA' or 'ONA' type
dataset.drop(dataset[dataset['Type']=='Music'].index, inplace=True)
dataset.drop(dataset[dataset['Type']=='Unknown'].index, inplace=True)
dataset['Type'].value_counts()

TV         4996
Movie      3031
OVA        2600
Special    2195
ONA        1886
Name: Type, dtype: int64

In [8]:
dataset.describe(include='all')

,MAL_ID,Name_x,Score_x,Genders_x,sypnopsis,Name_y,Score_y,Genders_y,English name,Japanese name,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
count,14708.000000,14708,14708,14708,14700,14708,14708,14708,14708,14708,...,14708,14708,14708,14708,14708,14708,14708,14708,14708,14708
unique,NaN,14704,530,4726,13762,14704,530,4726,6094,13888,...,3322,3596,4439,4836,4182,3268,2220,1498,1105,1072
top,NaN,Maou Gakuin no Futekigousha: Shijou Saikyou no...,Unknown,Comedy,No synopsis information has been added to this...,Maou Gakuin no Futekigousha: Shijou Saikyou no...,Unknown,Comedy,Unknown,Unknown,...,4.0,Unknown,Unknown,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,4.0
freq,NaN,3,4347,668,678,3,4347,668,8456,36,...,810,2526,1295,489,470,543,923,1206,1415,830
mean,20883.585328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,14759.012839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,5869.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,21710.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,34957.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Drop duplicates anime 
dataset.drop_duplicates(subset='Name_x', inplace=True)
dataset.shape

(14704, 39)

In [10]:
#Define content with no synopsis 
no_synopsis = dataset['sypnopsis'].mode()[0]
no_synopsis

'No synopsis information has been added to this title. Help improve our database by adding a synopsis here .'

In [11]:
dataset = dataset[(dataset['sypnopsis'].isnull()==False) & (dataset['sypnopsis'] != no_synopsis)]
dataset.shape

(14018, 39)

In [12]:
list(dataset.columns)

['MAL_ID',
 'Name_x',
 'Score_x',
 'Genders_x',
 'sypnopsis',
 'Name_y',
 'Score_y',
 'Genders_y',
 'English name',
 'Japanese name',
 'Type',
 'Episodes',
 'Aired',
 'Premiered',
 'Producers',
 'Licensors',
 'Studios',
 'Source',
 'Duration',
 'Rating',
 'Ranked',
 'Popularity',
 'Members',
 'Favorites',
 'Watching',
 'Completed',
 'On-Hold',
 'Dropped',
 'Plan to Watch',
 'Score-10',
 'Score-9',
 'Score-8',
 'Score-7',
 'Score-6',
 'Score-5',
 'Score-4',
 'Score-3',
 'Score-2',
 'Score-1']

In [13]:
dataset= dataset.drop(['Name_y',
 'Score_y',
 'Genders_y'], axis=1)

In [14]:
#drop anime with duplicate sypnopsis 
dataset.drop_duplicates(subset='sypnopsis', inplace=True)
dataset.describe(include='all')

,MAL_ID,Name_x,Score_x,Genders_x,sypnopsis,English name,Japanese name,Type,Episodes,Aired,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
count,13757.000000,13757,13757,13757,13757,13757,13757,13757,13757,13757,...,13757,13757,13757,13757,13757,13757,13757,13757,13757,13757
unique,NaN,13757,524,4636,13757,5909,13004,5,199,9681,...,3311,3578,4418,4808,4166,3259,2215,1494,1104,1069
top,NaN,Cowboy Bebop,Unknown,Comedy,"In the year 2071, humanity has colonized sever...",Unknown,Unknown,TV,1,Unknown,...,4.0,Unknown,Unknown,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,4.0
freq,NaN,1,3727,605,1,7712,26,4797,6046,227,...,724,2139,1083,417,381,429,746,965,1146,744
mean,20247.499382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,14712.436883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,5267.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,19997.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,34577.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#Take Synposis as a string
dataset['sypnopsis'] = dataset['sypnopsis'].astype(str)

In [16]:
# Initialize empty column
dataset['Keywords'] = ''

# function to get keywords from a text
def get_keywords(x):
    plot = x
    
    # initialize Rake using english stopwords from NLTK, and all punctuation characters
    r = Rake()
    
    # extract keywords from text
    r.extract_keywords_from_text(plot)
    
    # get dictionary with keywords and scores
    scores = r.get_word_degrees()
    
    # return new keywords as list, ignoring scores
    return(list(scores.keys()))

# Apply function to generate keywords
dataset['Keywords'] = dataset['sypnopsis'].apply(get_keywords)
dataset.head()

,MAL_ID,Name_x,Score_x,Genders_x,sypnopsis,English name,Japanese name,Type,Episodes,Aired,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Keywords
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,"[year, 2071, humanity, colonized, several, pla..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",...,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0,"[day, another, bounty, —, life, often, unlucky..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",...,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0,"[vash, stampede, man, $$, 60, 000, bounty, hea..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",...,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0,"[ches, individuals, special, powers, like, esp..."
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",...,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0,"[dark, century, people, suffering, rule, devil..."


In [17]:
# Split the features (genres) into list 
def tokenize(x):
    if isinstance(x, list):
        return [i.lower().split(", ") for i in x]
    else:
        if isinstance(x, str):
            return x.lower().split(", ")
        else:
            return ''   

dataset['Genders_x'] = dataset['Genders_x'].apply(tokenize)
dataset.head()

,MAL_ID,Name_x,Score_x,Genders_x,sypnopsis,English name,Japanese name,Type,Episodes,Aired,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Keywords
0,1,Cowboy Bebop,8.78,"[action, adventure, comedy, drama, sci-fi, space]","In the year 2071, humanity has colonized sever...",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,"[year, 2071, humanity, colonized, several, pla..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"[action, drama, mystery, sci-fi, space]","other day, another bounty—such is the life of ...",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",...,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0,"[day, another, bounty, —, life, often, unlucky..."
2,6,Trigun,8.24,"[action, sci-fi, adventure, comedy, drama, sho...","Vash the Stampede is the man with a $$60,000,0...",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",...,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0,"[vash, stampede, man, $$, 60, 000, bounty, hea..."
3,7,Witch Hunter Robin,7.27,"[action, mystery, police, supernatural, drama,...",ches are individuals with special powers like ...,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",...,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0,"[ches, individuals, special, powers, like, esp..."
4,8,Bouken Ou Beet,6.98,"[adventure, fantasy, shounen, supernatural]",It is the dark century and the people are suff...,Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",...,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0,"[dark, century, people, suffering, rule, devil..."


In [18]:
dataset_keys = pd.DataFrame() 

dataset_keys['title'] = dataset['Name_x']
dataset_keys['bag_of_words'] = ''

def bag_words(x):
    return(' '.join(x['Genders_x'])+ ' ' + ' '.join(x['Keywords']) + ' ' )
dataset_keys['bag_of_words'] = dataset.apply(bag_words, axis = 1)

dataset_keys.head()

,title,bag_of_words
0,Cowboy Bebop,action adventure comedy drama sci-fi space yea...
1,Cowboy Bebop: Tengoku no Tobira,action drama mystery sci-fi space day another ...
2,Trigun,action sci-fi adventure comedy drama shounen v...
3,Witch Hunter Robin,action mystery police supernatural drama magic...
4,Bouken Ou Beet,adventure fantasy shounen supernatural dark ce...


In [19]:
cv = CountVectorizer()
bow = cv.fit_transform(dataset_keys['bag_of_words'])

In [20]:
cosine_sim = cosine_similarity(bow, bow)

In [21]:
# create list of indices for later matching
indices = pd.Series(dataset_keys.index, index = dataset_keys['title'])

def recommend_anime(title, n = 10, cosine_sim = cosine_sim):
    movies = []
    
    # retrieve matching movie title index
    if title not in indices.index:
        print("Movie not in database.")
        return
    else:
        idx = indices[title]
    
    # cosine similarity scores of movies in descending order
    scores = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
    # top n most similar movies indexes
    # use 1:n because 0 is the same movie entered
    top_n_idx = list(scores.iloc[1:n].index)
        
    return pd.DataFrame(dataset_keys['title'].iloc[top_n_idx])

In [22]:
recommend_anime('One Punch Man')

,title
7828,Magical★Taruruuto-kun: Moeru! Yuujou no Mahou ...
7827,Magical★Taruruuto-kun: Sukisuki Takoyaki!
7805,Magical★Taruruuto-kun Movie
1312,Mahou Shoujo Pretty Sammy Specials
10519,Kono Subarashii Sekai ni Shukufuku wo! 2
7055,Shimajirou to Fufu no Daibouken Movie: Sukue! ...
623,Otogi Story Tenshi no Shippo Specials
3094,Onegai My Melody: Kuru Kuru Shuffle!
705,Mahoujin Guru Guru Movie


# **Review Based Recommendation**

In [23]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [24]:
anime_list = pd.read_csv('/content/anime.csv',usecols=['MAL_ID',"Name"])
display(anime_list.shape)
anime_list.head()

(17562, 2)

,MAL_ID,Name
0,1,Cowboy Bebop
1,5,Cowboy Bebop: Tengoku no Tobira
2,6,Trigun
3,7,Witch Hunter Robin
4,8,Bouken Ou Beet


In [25]:
#Import rating dataset
rating_data =pd.read_csv('/content/rating_complete.csv')
display(rating_data.shape)
rating_data.head()

(27920442, 3)

,user_id,anime_id,rating
0,0,430.0,9.0
1,0,1004.0,5.0
2,0,3010.0,7.0
3,0,570.0,7.0
4,0,2762.0,9.0


In [26]:
users_count = rating_data.groupby("user_id").size().reset_index()
users_count.columns = ["user_id", "anime_count"]
print('Numbers of unique users : ', users_count.shape[0])

#Filter the users
filtered_users = users_count[users_count.anime_count >= 500]
users = list(filtered_users.user_id)
print('Numbers of unique users with 500 or more anime ratings : ', len(users))

Numbers of unique users :  150758
Numbers of unique users with 500 or more anime ratings :  11111


In [27]:
rating_data = rating_data[rating_data['user_id'].isin(users)]
print ("rating shape:", rating_data.shape)
print (rating_data.info())

rating shape: (9197409, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9197409 entries, 1683 to 27920441
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   int64  
 1   anime_id  float64
 2   rating    float64
dtypes: float64(2), int64(1)
memory usage: 280.7 MB
None


In [ ]:
unique_users = {int(x): i for i,x in enumerate(rating_data.user_id.unique())}
unique_animes = {int(x): i for i,x in enumerate(anime_list.MAL_ID.unique())}

print(len(unique_animes), len(unique_users))
anime_collaborative_filter = np.zeros((len(unique_animes), len(unique_users)))

for user_id, MAL_ID, rating in rating_data.values:
  anime_collaborative_filter[unique_animes[MAL_ID], unique_users[user_id]] = rating

In [28]:
def get_recommended(title, n_neighbors=10):
    model_knn = NearestNeighbors(metric='cosine', n_neighbors=n_neighbors)
    model_knn.fit(csr_matrix(anime_collaborative_filter))
    
    query_index = anime_list[anime_list['Name']==title].index[0]

    distances, indices = model_knn.kneighbors(anime_collaborative_filter[query_index,:].reshape(1, -1), n_neighbors = n_neighbors)
    result = []
    for i in range(0, len(distances.flatten())):
        index = indices.flatten()[i]
        if index == query_index:
            continue
        result.append(anime_list.iloc[index])
        
    return pd.DataFrame(result)

In [32]:
get_recommended('One Punch Man')

,MAL_ID,Name
7449,16498,Shingeki no Kyojin
11185,31964,Boku no Hero Academia
8148,19815,No Game No Life
10677,30831,Kono Subarashii Sekai ni Shukufuku wo!
10876,31240,Re:Zero kara Hajimeru Isekai Seikatsu
6614,11757,Sword Art Online
4636,6547,Angel Beats!
7323,15809,Hataraku Maou-sama!
8292,20507,Noragami
